# Tensorflow2.0 小练习

In [32]:
import torch
import numpy as np
import torch.nn.functional as F

## 实现softmax函数

In [26]:
def softmax(x):
    ##########
    '''实现softmax函数，只要求对最后一维归一化，
    不允许用tf自带的softmax函数'''
    ##########
    # 步骤1: 对输入数组 x 的最后一个维度计算指数值
    exp_x =  np.exp(x)
    
    # 步骤2: 计算在同一行(即最后一个维度)上指数值的总和
    sum_exp_x = np.sum(exp_x, axis=-1, keepdims=True)
    
    # 步骤3: 将指数值除以对应行的指数值总和，得到归一化的概率
    prob_x = exp_x / sum_exp_x
    return prob_x

test_data = np.random.normal(size=[10, 5])
(softmax(test_data) - torch.softmax(torch.tensor(test_data), axis=-1).numpy())**2 <0.0001

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

## 实现sigmoid函数

In [27]:
def sigmoid(x):
    ##########
    '''实现sigmoid函数， 不允许用tf自带的sigmoid函数'''
    ##########
    prob_x = 1 / (1 + np.exp(-x))
    return prob_x

test_data = np.random.normal(size=[10, 5])
(sigmoid(test_data) - torch.sigmoid(torch.tensor(test_data)).numpy())**2 < 0.0001

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

## 实现 softmax 交叉熵loss函数

In [ ]:
def softmax(x):
    """计算Softmax."""
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))  # 减去最大值以防止溢出
    return e_x / np.sum(e_x, axis=-1, keepdims=True)

def softmax_ce(x, label):
    '''实现 softmax 交叉熵loss函数， 不允许用tf自带的softmax_cross_entropy函数'''
    
    # 计算softmax输出
    prob_x = softmax(x)
    
    # 确保label是one-hot编码或可以被当作索引使用
    if len(label.shape) == 1:  # 如果标签是索引
        batch_size = label.shape[0]
        losses = -np.log(prob_x[np.arange(batch_size), label])
    else:  # 如果标签是one-hot编码
        losses = -np.sum(label * np.log(prob_x + 1e-20), axis=1)  # 加一个小常数防止log(0)
    
    # 平均损失
    loss = np.mean(losses)
    
    return loss

test_data = np.random.normal(size=[10, 5])
label = np.zeros_like(test_data)
label[np.arange(10), np.random.randint(0, 5, size=10)]=1.

# 计算自定义softmax交叉熵损失
custom_loss = softmax_ce(test_data, label)

# 使用PyTorch计算交叉熵损失
pytorch_loss = F.cross_entropy(torch.tensor(test_data), torch.tensor(label), reduction='mean')
print(custom_loss, pytorch_loss.item())

# 比较两者差值平方是否小于0.0001
diff_square = (custom_loss - pytorch_loss.item()) ** 2 < 0.0001
print(diff_square)

2.0387167254743517 2.0387167254743517
True


## 实现 sigmoid 交叉熵loss函数

In [37]:
def sigmoid_ce(x, label):
    '''实现 sigmoid 交叉熵loss函数'''
    # 确保输入是numpy数组
    if not isinstance(x, np.ndarray):
        x = np.array(x)
    if not isinstance(label, np.ndarray):
        label = np.array(label)

    # 计算sigmoid值
    sigmoid_x = sigmoid(x)
    
    # 防止log(0)，对sigmoid_x进行裁剪
    epsilon = 1e-12
    sigmoid_x = np.clip(sigmoid_x, epsilon, 1. - epsilon) # np.clip 函数的作用是将 sigmoid_x 中小于 epsilon 的值设为 epsilon，大于 1 - epsilon 的值设为 1 - epsilon
    
    # 计算交叉熵损失
    loss = -np.mean(label * np.log(sigmoid_x) + (1 - label) * np.log(1 - sigmoid_x))
    
    return loss

test_data = np.random.normal(size=[10])
label = np.random.randint(0, 2, 10).astype(test_data.dtype)
print (label)

diff_square = ( F.binary_cross_entropy_with_logits(torch.tensor(test_data), torch.tensor(label), reduction='mean').item() 
               - sigmoid_ce(test_data, label))**2 < 0.0001
print(diff_square)


[0. 1. 0. 1. 1. 0. 0. 1. 0. 0.]
True
